In [45]:
import requests, datetime
from pymongo import MongoClient

In [85]:
client = MongoClient('mongodb://localhost:27018') #connect to sentiment db
db = client.test_db


In [86]:
#make sure there are no collections
db.list_collection_names()

[]

In [48]:
#if there's a test db, remove it
#drop collection
for collection in db.list_collection_names():
    if collection == 'test':
        db[collection].drop()

In [49]:
#make sure there are no collections
db.list_collection_names()

[]

In [41]:
collection = db.test

In [70]:
transformed_data = [
{'date': '04-27-2023', 'news': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.', 'entities': ['Bed Bath & Beyond', 'Sharper Image', 'Linens ’ n Things', 'Bed Bath & Beyond'], 'search_term': 'Stock Market', 'source': 'nyt'},
{'date': '04-23-2023', 'news': 'While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.', 'entities': ['Bed Bath & Beyond', 'Sharper Image', 'Linens ’ n Things', 'Bed Bath & Beyond'], 'search_term': 'Stock Market', 'source': 'nyt'},
{'date': '04-23-2023', 'news': 'Those who invested in International Business Machines (NYSE:IBM) five years ago are up 14%', 'entities': ['International Business Machines', 'IBM'], 'search_term': 'IBM', 'source': 'yfinance'},
{'date': '04-23-2023', 'news': 'Those who invested in International Business Machines (NYSE:IBM) 10 years ago are up 12224%', 'entities': ['International Business Machines', 'IBM'], 'search_term': 'IBM', 'source': 'yfinance'}

]

In [61]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [62]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [63]:
res = {'data': []}
for i in range(len(transformed_data)):
    inputs = tokenizer(transformed_data[i]['news'], padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()
    table = {'Headline':transformed_data[i]['news'],
        'positive': positive[0],
        'negative': negative[0],
        'neutral': neutral[0],
        'date': "04-23-2023",
        'search_term': transformed_data[i]['search_term'],
        'source': transformed_data[i]['source']
        }
    res['data'].append(table)
res['data']

[{'Headline': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.',
  'positive': 0.44310519099235535,
  'negative': 0.022097798064351082,
  'neutral': 0.5347970128059387,
  'date': '04-23-2023',
  'search_term': 'Stock Market',
  'source': 'nyt'},
 {'Headline': 'Those who invested in International Business Machines (NYSE:IBM) five years ago are up 14%',
  'positive': 0.8855618834495544,
  'negative': 0.015177002176642418,
  'neutral': 0.09926115721464157,
  'date': '04-23-2023',
  'search_term': 'IBM',
  'source': 'yfinance'},
 {'Headline': 'Those who invested in International Business Machines (NYSE:IBM) 10 years ago are up 12224%',
  'positive': 0.7396247386

In [64]:
data = {
    ''
}

In [71]:
from collections import defaultdict
data  = defaultdict(dict)

In [72]:
res = []
for i in range(len(transformed_data)):
    if transformed_data[i]['date'] not in data:
        data[ transformed_data[i]['date'] ] = {}
    if transformed_data[i]['search_term'] not in data[ transformed_data[i]['date'] ]:
        data[ transformed_data[i]['date'] ][ transformed_data[i]['search_term'] ] = {
            'avg_sentiment': [],
            'news': []
        }

    inputs = tokenizer(transformed_data[i]['news'], padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()
    sentiment_data = {'Headline':transformed_data[i]['news'],
        'sentiment': {
            'positive': positive[0],
            'negative': negative[0],
            'neutral': neutral[0] 
        },
        'source': transformed_data[i]['source']
    }
    data[ transformed_data[i]['date'] ][ transformed_data[i]['search_term'] ]['news'].append(sentiment_data)
data

defaultdict(dict,
            {'04-27-2023': {'Stock Market': {'avg_sentiment': [],
               'news': [{'Headline': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.',
                 'sentiment': {'positive': 0.44310519099235535,
                  'negative': 0.022097798064351082,
                  'neutral': 0.5347970128059387},
                 'search_term': 'Stock Market',
                 'source': 'nyt'}]}},
             '04-23-2023': {'Stock Market': {'avg_sentiment': [],
               'news': [{'Headline': 'While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquir

In [43]:
query = {
    "name": {'$eq': 'Stock Market'}
}

In [44]:
for doc in collection.find(query):
    print(doc)

{'_id': ObjectId('6446e2c98a9640333d7df86e'), 'name': 'Stock Market', 'data': [{'Headline': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.', 'positive': 0.44310519099235535, 'negative': 0.022097798064351082, 'neutral': 0.5347970128059387, 'date': '04-27-2023', 'search_term': 'Stock Market', 'source': 'nyt'}]}


In [101]:
client = MongoClient('mongodb://localhost:27018') #connect to sentiment db
db = client.test_db


In [102]:
#make sure there are no collections
db.list_collection_names()

[]

In [103]:
#if there's a test db, remove it
#drop collection
for collection in db.list_collection_names():
    if collection == 'test':
        db[collection].drop()

In [104]:
#make sure there are no collections
db.list_collection_names()

[]

In [105]:
collection = db.test

In [106]:
#call endpoint to run sentiment
#make sure the extraction flask app is listening on port 8001
url = 'http://localhost:8002/sentiment'

response = requests.get(url)

print(response.json())

{'message': 'Data push to sentiment DB- Succeded'}


In [107]:
for doc in collection.find():
    print(doc)

{'_id': ObjectId('6446fa468a9640333d7e0271'), 'date': '04-23-2023', 'data': {'04-23-2023': {'Stock Market': {'avg_sentiment': [], 'news': [{'Headline': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.', 'sentiment': {'positive': 0.44310519099235535, 'negative': 0.022097798064351082, 'neutral': 0.5347970128059387}, 'source': 'nyt'}]}, 'IBM': {'avg_sentiment': [], 'news': [{'Headline': 'Those who invested in International Business Machines (NYSE:IBM) five years ago are up 14%', 'sentiment': {'positive': 0.8855618834495544, 'negative': 0.015177002176642418, 'neutral': 0.09926115721464157}, 'source': 'yfinance'}]}}}}
